In [91]:
#using BenchmarkTools
srand(280) # seed
X = rand(0.0:2.0, 1000, 10000)

1000×10000 Array{Float64,2}:
 0.0  0.0  1.0  2.0  2.0  2.0  2.0  0.0  …  0.0  1.0  1.0  0.0  2.0  2.0  1.0
 2.0  2.0  1.0  2.0  1.0  0.0  1.0  2.0     2.0  2.0  0.0  0.0  0.0  0.0  2.0
 1.0  2.0  0.0  0.0  2.0  1.0  0.0  1.0     1.0  2.0  1.0  0.0  1.0  0.0  0.0
 0.0  2.0  0.0  2.0  2.0  0.0  1.0  1.0     1.0  2.0  0.0  0.0  1.0  0.0  2.0
 1.0  2.0  0.0  2.0  0.0  0.0  0.0  0.0     0.0  1.0  2.0  0.0  0.0  1.0  0.0
 2.0  1.0  1.0  2.0  1.0  0.0  2.0  0.0  …  1.0  2.0  0.0  1.0  1.0  2.0  2.0
 1.0  2.0  1.0  1.0  1.0  1.0  2.0  0.0     2.0  0.0  2.0  1.0  0.0  1.0  1.0
 2.0  0.0  2.0  2.0  0.0  2.0  2.0  2.0     1.0  0.0  2.0  0.0  2.0  0.0  2.0
 1.0  1.0  0.0  0.0  0.0  2.0  0.0  1.0     2.0  2.0  1.0  1.0  2.0  1.0  1.0
 2.0  2.0  1.0  1.0  0.0  2.0  1.0  2.0     0.0  2.0  0.0  0.0  2.0  1.0  2.0
 1.0  2.0  2.0  0.0  1.0  2.0  1.0  1.0  …  2.0  1.0  1.0  1.0  2.0  1.0  2.0
 0.0  2.0  2.0  1.0  1.0  2.0  0.0  1.0     0.0  2.0  1.0  1.0  0.0  0.0  0.0
 2.0  2.0  1.0  0.0  2.0  0.0  2.0 

In [27]:
using BenchmarkTools

In [80]:
function naive(X::Matrix{Float64})
    n = size(X, 1)
    m = size(X, 2)
    ematrix = Matrix{Float64}(n, n)
    phi = Matrix{Float64}(n, n)
    p = sum(X, 1) / (2.0 * n)
    pnorm = norm(p)^2
    pnorm2 = norm(1 - p)^2
    for j = 1:n, k = 1:m, i = 1:n 
        @fastmath @inbounds ematrix[i, j] = ematrix[i, j] + X[i, k] * X[j, k] + (2.0 - X[i, k]) * (2.0 - X[j, k])
    end
    ematrix = ematrix ./ 4.0
    for j = 1:n, i = 1:n 
        phi[i, j] = (ematrix[i, j] - pnorm - pnorm2) / (m - pnorm - pnorm2)
    end
    phi
end

naive (generic function with 1 method)

Array{Float64, 2}) in module Main at In[78]:2 overwritten at In[80]:2.


In [92]:
naive(X)

1000×1000 Array{Float64,2}:
  0.667981     -0.000159068  -0.00316105  …  -0.00376144   0.00314311
 -0.000159068   0.674786      0.00134192      0.0118488   -0.00166006
 -0.00316105    0.00134192    0.668682        0.00514443  -0.00386151
 -0.00436184    0.00014113   -0.00286085      0.00864673  -0.00436184
  0.00144199   -0.00666335   -0.0101657       0.00304304   0.00344331
 -0.0116667    -0.00216039    0.00214245  …  -0.00416171  -0.00396157
  0.000641459   0.00394364    0.00834654      0.00174218   0.00124186
 -0.00876474    0.00014113   -0.00766401     -0.00105966   0.00214245
  0.000641459  -0.00866467   -0.00386151     -0.00216039   0.0148508 
  0.00964739   -0.00426177    0.0141504       0.00304304   0.00924713
 -0.00586283   -0.00516237    0.00144199  …   0.00414377  -0.00386151
 -0.0044619    -0.00456197    0.00864673      0.00304304   0.00754601
 -0.00296091   -0.00386151   -0.013768        0.00224251  -0.00696355
  ⋮                                       ⋱                   

In [93]:
@benchmark naive(X)

BenchmarkTools.Trial: 
  memory estimate:  23.12 MiB
  allocs estimate:  12
  --------------
  minimum time:     13.061 s (0.00% GC)
  median time:      13.061 s (0.00% GC)
  mean time:        13.061 s (0.00% GC)
  maximum time:     13.061 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

## Q2

0. Show the **Sherman-Morrison formula**
$$
	(\mathbf{A} + \mathbf{u} \mathbf{u}^T)^{-1} = \mathbf{A}^{-1} - \frac{1}{1 + \mathbf{u}^T \mathbf{A}^{-1} \mathbf{u}} \mathbf{A}^{-1} \mathbf{u} \mathbf{u}^T \mathbf{A}^{-1},
$$
where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is nonsingular and $\mathbf{u} \in \mathbb{R}^n$. This formula supplies the inverse of the symmetric, rank-one  perturbation of $\mathbf{A}$.

0. Show the **Woodbury formula**
$$
	(\mathbf{A} + \mathbf{U} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$
where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is nonsingular, $\mathbf{U}, \mathbf{V} \in \mathbb{R}^{n \times m}$, and $\mathbf{I}_m$ is the $m \times m$ identity matrix. In many applications $m$ is much smaller than $n$. Woodbury formula generalizes Sherman-Morrison and is valuable because the smaller matrix $\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U}$ is cheaper to invert than the larger matrix $\mathbf{A} + \mathbf{U} \mathbf{V}^T$.

0. Show the **binomial inversion formula**
$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$
where $\mathbf{A}$ and $\mathbf{B}$ are nonsingular.

0. Show the identity
$$
	\text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) = \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U}).
$$
This formula is useful for evaluating the density of a multivariate normal with covariance matrix $\mathbf{A} + \mathbf{U} \mathbf{V}^T$.

1. Assume $(\mathbf{A} + \mathbf{uu}^{T})$ is invertible. I will show the Sherman-Morrison formula satisfies the properties of an inverse.
$$
\begin{align}
(\mathbf{A} + \mathbf{uu}^{T})\bigg(\mathbf{A}^{-1} - \frac{1}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\bigg) &=\mathbf{I} - \frac{\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} + \mathbf{uu}^{T}A^{-1}\mathbf{} -\frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1 + \mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I} - \frac{\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} + \frac{\mathbf{uu}^{T}A^{-1}+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} -\frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1 + \mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I} + \frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}-\frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1 + \mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I}
\end{align}
$$
This shows the Sherman-Morrison formula is a right-inverse. I need to check it's a left-inverse as well.
$$
\begin{align}
\bigg(\mathbf{A}^{-1} - \frac{1}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\bigg) (\mathbf{A} + \mathbf{uu}^{T})&=\mathbf{I} + \mathbf{A}^{-1}\mathbf{uu}^{T} - \frac{\mathbf{A}^{-1}\mathbf{uu}^{T}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} - \frac{\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I} + \frac{\mathbf{A}^{-1}\mathbf{uu}^{T}+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}\mathbf{A}^{-1}\mathbf{uu}^{T}-\mathbf{A}^{-1}\mathbf{uu}^{T}-\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&= \mathbf{I}
\end{align}
$$

 By definition of inverse,  $(\mathbf{A} + \mathbf{uu}^{T})^{-1}=\mathbf{A}^{-1} - \displaystyle\frac{1}{1 + \mathbf{u}^T \mathbf{A}^{-1} \mathbf{u}} \mathbf{A}^{-1} \mathbf{u} \mathbf{u}^T \mathbf{A}^{-1}$
 
2. In a similar manner,
$$
\begin{align}
(\mathbf{A} + \mathbf{UV}^{T})(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})&=\mathbf{I_n} + \mathbf{UV}^{T}\mathbf{A}^{-1}-\mathbf{U}(\mathbf{I_m}
+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1} - \mathbf{UV}^{T}\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UV}^{T}\mathbf{A}^{-1}-(\mathbf{U}+\mathbf{UV}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n} + \mathbf{UV}^{T}\mathbf{A}^{-1}-\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UV}^{T}\mathbf{A}^{-1}-\mathbf{U}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}
\end{align}
$$
 
 This shows the Woodbury formula is a right inverse for $\mathbf{A} + \mathbf{UV}^{T}$. To show it's a left inverse:
$$
\begin{align}
(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})(\mathbf{A} + \mathbf{UV}^{T})&= \mathbf{I_n} + \mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UV}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{V}^{T}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UV}^{T})\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})\mathbf{V}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{UV}^{T}\\
&=\mathbf{I_n}
\end{align}
$$

 By definition of inverse, $(\mathbf{A} + \mathbf{UV}^{T})^{-1}=(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})$
 
3. Continuing, 
$$
\begin{align}
(\mathbf{A}+\mathbf{UBV}^{T})(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})&= \mathbf{I_n}-\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-\mathbf{UBV}^{T}\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-(\mathbf{U}+\mathbf{UBV}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-\mathbf{UB}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-\mathbf{UBV}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}
\end{align}
$$

 Left inverse:
$$
\begin{align}
(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})(\mathbf{A}+\mathbf{UBV}^{T})&=\mathbf{I_n} + \mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UBV}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{V}^{T}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UBV}^{T})\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})\mathbf{BV}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{UBV}^{T}\\
&=\mathbf{I_n}
\end{align}
$$
 This shows $(\mathbf{A}+\mathbf{UBV}^{T})^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1}$
 
4. Observate that
 $$
\begin{align}
\begin{pmatrix}\mathbf{I_n} & \mathbf{U}\\\mathbf{0_{m\times n}} & \mathbf{I_m}+\mathbf{V}^{T}\mathbf{U}\end{pmatrix} = \begin{pmatrix}\mathbf{I_n} & \mathbf{0_{n\times m}}\\\mathbf{V}^{T} & \mathbf{I_m}\end{pmatrix}\begin{pmatrix}\mathbf{I_n}+\mathbf{U}\mathbf{V}^{T} & \mathbf{U}\\\mathbf{0_{m\times n}} & \mathbf{I_m}\end{pmatrix}\begin{pmatrix}\mathbf{I_n} & \mathbf{0_{n\times m}}\\-\mathbf{V}^{T} & \mathbf{I_m}\end{pmatrix}
\end{align}
$$

 The determinant of a product is the product of determinants. Thus, $\text{det}(\mathbf{I_n} + \mathbf{UV}^{T}) = \text{det}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{U})$
 
 Therefore
$$
\begin{align}
\text{det}(\mathbf{A}+\mathbf{U}\mathbf{V}^{T}) &=\text{det}(\mathbf{A}+\mathbf{A}\mathbf{A}^{-1}\mathbf{UV}^{T})\\
&=\text{det}(\mathbf{A})\text{det}(\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T})\\
&=\text{det}(\mathbf{A})\text{det}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})
\end{align}
$$

## Q3

Consider a mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} \in \mathbb{R}^n$, $\mathbf{X} \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y, Z, σ0, σ1),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```

1. From the problem formulation, $\mathbf{Z}\gamma \sim N_n(0, \mathbf{Z}\sigma^2_1\mathbf{Z}^{T})$ and $\varepsilon \sim N_n(0,\sigma^2_0\mathbf{I_n})$ with $\mathbf{Z}\gamma$ independent of $\varepsilon$

 Let $\tilde{\varepsilon} = \mathbf{Z}\gamma + \varepsilon$. This is the sum of two independent multivariate normal variables, so $\tilde{\varepsilon} \sim N_n(0, \sigma^2_0\mathbf{I_n} + \sigma^2_1\mathbf{Z}\mathbf{Z}^{T})$
 
 Therefore $\mathbf{y} = \mathbf{X}\beta + \tilde{\varepsilon} \sim N(\mathbf{X}\beta, \sigma^2_0\mathbf{I_n} + \sigma^2_1\mathbf{Z}\mathbf{Z}^{T})$

In [38]:
# Question 3, part 2

using BenchmarkTools, Distributions

srand(280)
n, q = 2000, 10
Z = randn(n, q)
σ0, σ1 = 0.5, 2.0
Σ = σ1^2 * Z * Z.' + σ0^2 * I
mvn = MvNormal(Σ) # MVN(0, Σ)
y = rand(mvn); # generate one instance from MNV(0, Σ)

In [43]:
function logpdf_mvn(y::Array{Float64,1}, Z::Array{Float64,2}, σ0::Float64, σ1::Float64)
    k = length(y)
    
    # Save the cholesky decomposition of Σ. Cholesky decompositions are more computationally efficient
    Σchol = cholfact(Σ)
    
    # The // represent fractions in Julia.
    # The :L syntax means I want the lower triangular matrix in the cholesky decomposition
    - (k//2) * log(2π) - (1//2) * logdet(Σchol) - (1//2) * sumabs2(Σchol[:L] \ y)

end

logpdf_mvn (generic function with 2 methods)

Array{Float64, 1}, Array{Float64, 2}, Float64, Float64) in module Main at In[39]:2 overwritten at In[43]:2.


In [44]:
@show logpdf_mvn(y, Z, σ0, σ1)
@show logpdf(mvn, y)

logpdf_mvn(y,Z,σ0,σ1) = -1571.5736734654183
logpdf(mvn,y) = -1571.5736734654183


-1571.5736734654183

In [45]:
@benchmark logpdf_mvn(y, Z, σ0, σ1)

BenchmarkTools.Trial: 
  memory estimate:  61.05 MiB
  allocs estimate:  25
  --------------
  minimum time:     85.209 ms (0.76% GC)
  median time:      91.932 ms (4.92% GC)
  mean time:        94.465 ms (4.77% GC)
  maximum time:     125.584 ms (3.51% GC)
  --------------
  samples:          53
  evals/sample:     1

In [42]:
@benchmark logpdf(mvn, y)

BenchmarkTools.Trial: 
  memory estimate:  15.78 KiB
  allocs estimate:  3
  --------------
  minimum time:     5.843 ms (0.00% GC)
  median time:      5.962 ms (0.00% GC)
  mean time:        6.014 ms (0.00% GC)
  maximum time:     7.061 ms (0.00% GC)
  --------------
  samples:          826
  evals/sample:     1

In [56]:
Σchol = cholfact(Σ);
Σchol[:L] * Σchol[:U]
Σ

2000×2000 Array{Float64,2}:
  48.4398     12.5751      -1.85451   …   -0.963463     7.73531     5.55813 
  12.5751     60.0283     -43.9952         0.0624533    1.79251     8.32576 
  -1.85451   -43.9952      63.5581        -1.66745      5.60657     3.39898 
   7.47823    -6.66154      4.66697        3.53446     -1.86866     6.24601 
   3.18518    13.4341     -10.1183       -14.0829       0.217011    1.42379 
   4.58176     8.38486     14.3487    …    2.26277      1.89618    14.042   
  -9.12281   -18.4289       1.57148      -13.0892      -1.0412     -7.12815 
  -4.47219    17.7271     -30.5422         2.22316    -12.2928     -2.29981 
  -5.18916    15.2756      -2.30837       -3.42225      0.526121    5.61582 
 -10.7986      8.64372    -12.7132         3.48297     -9.16786     0.784319
 -17.8025     16.4128     -29.3033    …   13.6395     -12.5796     -5.20061 
  11.7025     -3.62996     -3.01955       -6.09396     -2.95715     1.75864 
  11.4505     16.6458      -0.157436       1.198